# Pipeline Corrosion Predictive Maintenance - Enhanced Analysis

## Objectives
1. **Data Integration Strategy**: Analyze whether to merge datasets vs keep separate
2. **Comprehensive EDA**: Univariate → Bivariate → Multivariate analysis
3. **Feature Engineering**: Domain-specific features (corrosion rate, remaining life, risk scoring)
4. **Predictive Modeling**: Multi-class classification (Critical/Moderate/Normal condition)
5. **Model Interpretability**: SHAP analysis for actionable insights

## Reference Flow
Following **Business-Analytics-Assignment** structured approach:
- `src/ingest.py` → Load and validate data
- `src/aggregate.py` → Merge and feature engineering
- `src/preprocess.py` → Scaling, encoding, feature selection
- `src/train.py` → Model training with hyperparameter tuning
- EDA helpers for systematic analysis

---

In [ ]:
# Import necessary libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import lightgbm as lgb
import shap

# Set styles
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

print("✅ Libraries imported successfully")

In [ ]:
# Compare schemas
corrosion_cols = set(pipeline_corrosion.columns)
bearing_cols = set(bearing_features.columns)

common_cols = corrosion_cols.intersection(bearing_cols)
corrosion_only = corrosion_cols - bearing_cols
bearing_only = bearing_cols - corrosion_cols

print("Schema Comparison:")
print(f"\nCommon Columns ({len(common_cols)}): {common_cols}")
print(f"\nCorrosion-Only Columns ({len(corrosion_only)}):")
print(f"   {corrosion_only}")
print(f"\nBearing-Only Columns ({len(bearing_only)}):")
print(f"   {bearing_only}")

# Simulate merge impact
if len(common_cols) > 0:
    simulated_merge = pd.merge(
        pipeline_corrosion.head(100),
        bearing_features.head(100),
        on=list(common_cols),
        how='outer',
        indicator=True
    )
    
    total_cells = simulated_merge.shape[0] * simulated_merge.shape[1]
    nan_cells = simulated_merge.isna().sum().sum()
    nan_percent = (nan_cells / total_cells) * 100
    
    print(f"\nMerge Impact Simulation (first 100 rows each):")
    print(f"   Merged shape: {simulated_merge.shape}")
    print(f"   NaN percentage: {nan_percent:.1f}%")
    print(f"\n   Merge source breakdown:")
    print(simulated_merge['_merge'].value_counts())
else:
    print("\nNO common columns → CANNOT merge without creating synthetic join key")
    print("   Recommendation: KEEP SEPARATE and use multi-layer architecture")

In [ ]:
# Check for duplicates
duplicates = pipeline_corrosion.duplicated().sum()
print(f" Duplicate rows: {duplicates}")

# Check for missing values
missing = pipeline_corrosion.isnull().sum()
print(f"\n Missing values by column:")
print(missing[missing > 0] if missing.sum() > 0 else " No missing values")

# Check unique values for categorical columns
categorical_cols = pipeline_corrosion.select_dtypes(include=['object']).columns
print(f"\n Categorical columns:")
for col in categorical_cols:
    unique_count = pipeline_corrosion[col].nunique()
    print(f"  {col}: {unique_count} unique values")
    print(f"    → {pipeline_corrosion[col].unique()}")

In [ ]:
# Correlation heatmap
plt.figure(figsize=(12, 10))
correlation_matrix = pipeline_corrosion[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Numeric Features', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Identify highly correlated pairs (>0.7 or <-0.7)
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            high_corr_pairs.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                correlation_matrix.iloc[i, j]
            ))

print("\n Highly Correlated Feature Pairs (|r| > 0.7):")
if high_corr_pairs:
    for feat1, feat2, corr in high_corr_pairs:
        print(f"  {feat1} <-> {feat2}: {corr:.3f}")
else:
    print("  No multicollinearity issues detected")

In [ ]:
# Standardize features (important for LightGBM performance)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print(" Feature scaling complete")
print(f"   Mean of scaled train set: {X_train_scaled.mean().mean():.4f}")
print(f"   Std of scaled train set: {X_train_scaled.std().mean():.4f}")

In [ ]:
# Create model directory
model_dir = BASE_DIR / "models"
model_dir.mkdir(exist_ok=True)

# Save model
import joblib
model_path = model_dir / "corrosion_classifier.pkl"
joblib.dump(model, model_path)
print(f" Model saved to: {model_path}")

# Save feature importance
feature_importance_path = model_dir / "corrosion_feature_importance.csv"
feature_importance.to_csv(feature_importance_path, index=False)
print(f" Feature importance saved to: {feature_importance_path}")

# Save scaler and encoder
scaler_path = model_dir / "corrosion_scaler.pkl"
encoder_path = model_dir / "corrosion_label_encoder.pkl"
joblib.dump(scaler, scaler_path)
joblib.dump(le_target, encoder_path)
print(f" Scaler saved to: {scaler_path}")
print(f" Label encoder saved to: {encoder_path}")

print("\n Analysis Complete! All artifacts saved for deployment.")

## VIII. Save Artifacts

Save model and results for deployment

In [ ]:
# Generate insights report
print("="*70)
print(" "*15 + "PIPELINE CORROSION ANALYSIS INSIGHTS")
print("="*70)

print("\n1 DATA INTEGRATION STRATEGY:")
print("    Keep equipment types SEPARATE (corrosion vs bearing)")
print("    Only merge supplementary data (weather, operational context)")
print("    ❌ DO NOT merge all into 1 table → creates 80%+ NaN")

print("\n2 CONDITION DISTRIBUTION:")
condition_dist = pipeline_corrosion['Condition'].value_counts(normalize=True) * 100
for condition, pct in condition_dist.items():
    print(f"   {condition}: {pct:.1f}%")

print("\n3 TOP RISK FACTORS (from feature importance):")
for idx, row in feature_importance.head(5).iterrows():
    print(f"   {idx+1}. {row['feature']}: {row['importance']:.4f}")

print("\n4 MODEL PERFORMANCE:")
print(f"   Test Accuracy: {(y_pred_test == y_test).mean():.1%}")
print(f"   Critical Condition Detection: Check confusion matrix above")

print("\n5 ACTIONABLE RECOMMENDATIONS:")
print("   Prioritize pipelines with:")
print("      - High corrosion_rate_mm_year (>1.5 mm/year)")
print("      - Low safety_margin_percent (<40%)")
print("      - High pressure_thickness_ratio")
print("   Schedule inspection for pipelines with remaining_life <2 years")
print("   Consider material upgrade for high-loss-rate segments")

print("\n6 NEXT STEPS:")
print("   Deploy this model for real-time condition monitoring")
print("   Integrate with dashboard (equipment_summary.csv)")
print("   Set up automated alerts for Critical condition predictions")
print("   Continue with pump_pipeline and turbine_pipeline development")
print("\n" + "="*70)

## VII. Insights & Recommendations

### 7.1 Key Findings from EDA

Based on comprehensive analysis:

In [ ]:
# Create SHAP explainer
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test_scaled)

# Summary plot - overall feature impact
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test_scaled, class_names=le_target.classes_, show=False)
plt.title('SHAP Summary Plot - Feature Impact on Predictions', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print(" SHAP Analysis Complete")
print("\n Insights:")
print("  - Red points: High feature value")
print("  - Blue points: Low feature value")
print("  - X-axis: SHAP value (impact on model output)")

## VI. Model Interpretability - SHAP Analysis

Following best practices for explainable AI in PdM

In [ ]:
# Extract feature importance
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

# Plot top 15 features
plt.figure(figsize=(10, 8))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'], align='center')
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance Score')
plt.title('Top 15 Most Important Features', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print(" Top 10 Most Important Features:")
print(feature_importance.head(10).to_string(index=False))

### 5.3 Feature Importance Analysis

In [ ]:
# Classification report
print(" Classification Report (Test Set):\n")
print(classification_report(y_test, y_pred_test, target_names=le_target.classes_))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le_target.classes_,
            yticklabels=le_target.classes_)
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

print("\n Model Evaluation Complete")

### 5.2 Model Evaluation

In [ ]:
# Train LightGBM classifier
model = lgb.LGBMClassifier(
    objective='multiclass',
    num_class=3,
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    num_leaves=31,
    random_state=42,
    verbose=-1
)

model.fit(X_train_scaled, y_train)

# Predictions
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

print(" Model Training Complete")
print(f"\n Training Accuracy: {(y_pred_train == y_train).mean():.4f}")
print(f" Test Accuracy: {(y_pred_test == y_test).mean():.4f}")

## V. Model Training - LightGBM Classifier

Following reference workflow: `src/train.py` with hyperparameter tuning

### 5.1 Train Baseline Model

### 4.3 Feature Scaling

In [ ]:
# Select features for modeling
feature_cols = [col for col in df_encoded.columns 
                if col not in ['Condition', 'Condition_encoded'] 
                and df_encoded[col].dtype in [np.float64, np.int64, np.uint8]]

X = df_encoded[feature_cols]
y = df_encoded['Condition_encoded']

print(f" Feature Set:")
print(f"  Features: {len(feature_cols)}")
print(f"  Samples: {X.shape[0]}")
print(f"  Target distribution: {y.value_counts().to_dict()}")

# Train-test split (70-30)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\n Data Split:")
print(f"  Train: {X_train.shape[0]} samples")
print(f"  Test: {X_test.shape[0]} samples")
print(f"  Train class distribution: {y_train.value_counts().to_dict()}")
print(f"  Test class distribution: {y_test.value_counts().to_dict()}")

### 4.2 Feature Selection and Train-Test Split

In [ ]:
# Label encode target variable
le_target = LabelEncoder()
df['Condition_encoded'] = le_target.fit_transform(df['Condition'])

print(" Target Variable Encoding:")
for idx, label in enumerate(le_target.classes_):
    print(f"  {label} → {idx}")

# One-hot encode other categorical features
categorical_features = ['Material', 'Grade', 'loss_rate_severity']
df_encoded = pd.get_dummies(df, columns=categorical_features, drop_first=True)

print(f"\n After encoding, DataFrame shape: {df_encoded.shape}")
print(f"   Original: {df.shape[1]} columns → Encoded: {df_encoded.shape[1]} columns")

## IV. Data Preprocessing for Modeling

Following reference workflow: `src/preprocess.py`

### 4.1 Encode Categorical Variables

In [ ]:
# Check engineered features by condition
print(" Engineered Features by Condition:\n")

key_engineered = ['corrosion_rate_mm_year', 'remaining_life_years', 
                  'safety_margin_percent', 'pressure_thickness_ratio']

for feature in key_engineered:
    print(f"\n{feature}:")
    print(df.groupby('Condition')[feature].agg(['mean', 'std', 'min', 'max']).round(2))

# Visualize engineered features vs condition
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, feature in enumerate(key_engineered):
    df.boxplot(column=feature, by='Condition', ax=axes[idx])
    axes[idx].set_title(f'{feature} by Condition')
    axes[idx].set_xlabel('Condition')
    axes[idx].set_ylabel(feature)
    plt.sca(axes[idx])
    plt.xticks(rotation=45)

plt.suptitle('')
plt.tight_layout()
plt.show()

print("\n Engineered features show clear separation between conditions")

### 3.2 Validate Engineered Features

In [ ]:
# Create working copy
df = pipeline_corrosion.copy()

# 1. Corrosion Rate (mm/year)
df['corrosion_rate_mm_year'] = df['Thickness_Loss_mm'] / df['Time_Years']

# 2. Remaining Thickness
df['remaining_thickness_mm'] = df['Thickness_mm'] - df['Thickness_Loss_mm']

# 3. Remaining Life Estimate (years) - assuming min safe thickness = 30% of original
min_safe_thickness = df['Thickness_mm'] * 0.3
df['remaining_life_years'] = (df['remaining_thickness_mm'] - min_safe_thickness) / df['corrosion_rate_mm_year']
df['remaining_life_years'] = df['remaining_life_years'].clip(lower=0)  # Cannot be negative

# 4. Safety Margin (%)
df['safety_margin_percent'] = (df['remaining_thickness_mm'] / df['Thickness_mm']) * 100

# 5. Pressure to Thickness Ratio (risk indicator)
df['pressure_thickness_ratio'] = df['Max_Pressure_psi'] / df['remaining_thickness_mm']

# 6. Loss Rate Severity (categorical from corrosion rate)
def classify_loss_rate(rate):
    if rate < 0.5:
        return 'Low'
    elif rate < 1.5:
        return 'Moderate'
    else:
        return 'High'

df['loss_rate_severity'] = df['corrosion_rate_mm_year'].apply(classify_loss_rate)

print(" Feature Engineering Complete")
print(f"\n New Features Created:")
print(f"  - corrosion_rate_mm_year")
print(f"  - remaining_thickness_mm")
print(f"  - remaining_life_years")
print(f"  - safety_margin_percent")
print(f"  - pressure_thickness_ratio")
print(f"  - loss_rate_severity")
print(f"\n Updated DataFrame Shape: {df.shape}")

## III. Feature Engineering - Domain-Specific Features

Following reference workflow: `src/aggregate.py` → feature engineering based on domain knowledge

### 3.1 Compute Corrosion-Specific Features

### 2.5 Multivariate Analysis - Correlation Matrix

In [ ]:
# Key features vs Condition boxplots
key_features = ['Thickness_mm', 'Material_Loss_Percent', 'Time_Years', 'Max_Pressure_psi']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, feature in enumerate(key_features):
    pipeline_corrosion.boxplot(column=feature, by='Condition', ax=axes[idx])
    axes[idx].set_title(f'{feature} by Condition')
    axes[idx].set_xlabel('Condition')
    axes[idx].set_ylabel(feature)
    plt.sca(axes[idx])
    plt.xticks(rotation=45)

plt.suptitle('')  # Remove default title
plt.tight_layout()
plt.show()

print(" Bivariate Analysis: Key Features vs Condition")

In [ ]:
# Condition distribution
plt.figure(figsize=(10, 6))
condition_counts = pipeline_corrosion['Condition'].value_counts()
plt.pie(condition_counts, labels=condition_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Pipeline Condition Distribution', fontsize=14, fontweight='bold')
plt.show()

print(" Condition Distribution:")
print(condition_counts)
print(f"\n Class imbalance ratio: {condition_counts.max() / condition_counts.min():.2f}")

### 2.4 Bivariate Analysis - Target Variable Relationships

**Target**: `Condition` (Critical / Moderate / Normal)

Analyze how numeric features relate to pipeline condition

In [ ]:
# Boxplots to identify outliers
fig, axes = plt.subplots(n_rows, 3, figsize=(15, n_rows*4))
axes = axes.flatten()

for idx, col in enumerate(numeric_cols):
    axes[idx].boxplot(pipeline_corrosion[col].dropna(), vert=False)
    axes[idx].set_title(f'Boxplot of {col}', fontsize=10, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].grid(axis='x', alpha=0.3)

for idx in range(n_cols, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print(" Boxplot Analysis Complete - Check for outliers")

In [ ]:
# Histograms for all numeric columns
numeric_cols = pipeline_corrosion.select_dtypes(include=[np.number]).columns
n_cols = len(numeric_cols)
n_rows = (n_cols + 2) // 3

fig, axes = plt.subplots(n_rows, 3, figsize=(15, n_rows*4))
axes = axes.flatten()

for idx, col in enumerate(numeric_cols):
    axes[idx].hist(pipeline_corrosion[col].dropna(), bins=30, edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribution of {col}', fontsize=10, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(axis='y', alpha=0.3)

# Hide extra subplots
for idx in range(n_cols, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print(" Distribution Analysis Complete")

### 2.3 Univariate Analysis - Distributions

**Objective**: Understand distribution of each numeric feature để detect outliers và data quality issues

### 2.2 Data Quality Check

In [ ]:
# Display first few rows
print(" First 5 rows of Pipeline Corrosion Dataset:")
print(pipeline_corrosion.head())

print("\n Dataset Shape:", pipeline_corrosion.shape)
print("\n Column Names and Data Types:")
print(pipeline_corrosion.dtypes)

print("\n Summary Statistics:")
print(pipeline_corrosion.describe())

## II. Focus on Pipeline Corrosion Data - Comprehensive EDA

Following structured EDA workflow từ `Business-Analytics-Assignment/EDA_compile.ipynb`

### 2.1 Data Overview

### 1.2 Schema Comparison: Pipeline vs Bearing

**Phân tích**: Nếu merge corrosion + bearing vào 1 table, sẽ có bao nhiêu % NaN?

In [ ]:
# Define base paths
BASE_DIR = Path("d:/Final BA2")
CONVERTED_DIR = BASE_DIR / "converted_data"
FEATURES_DIR = BASE_DIR / "data/features"
METADATA_DIR = BASE_DIR / "data/metadata"

# Load primary pipeline corrosion dataset
pipeline_corrosion = pd.read_csv(CONVERTED_DIR / "processed/market_pipe_thickness_loss_dataset_clean.csv")

# Load supplementary metadata
equipment_master = pd.read_csv(METADATA_DIR / "equipment_master.csv")
weather_data = pd.read_csv(METADATA_DIR / "weather_data.csv")
operational_context = pd.read_csv(METADATA_DIR / "operational_context.csv")

# Load bearing features for comparison (to analyze merge feasibility)
bearing_features = pd.read_csv(FEATURES_DIR / "bearing_features.csv")

print(" Datasets loaded:")
print(f"  - Pipeline Corrosion: {pipeline_corrosion.shape}")
print(f"  - Equipment Master: {equipment_master.shape}")
print(f"  - Weather Data: {weather_data.shape}")
print(f"  - Operational Context: {operational_context.shape}")
print(f"  - Bearing Features (for comparison): {bearing_features.shape}")

### 1.1 Load All Available Datasets

Kiểm tra tất cả các file CSV để hiểu cấu trúc và khả năng merge

## I. Data Overview and Integration Analysis

### Key Questions:
1. **Số lượng bản ghi**: Mỗi dataset có bao nhiêu rows?
2. **Schema compatibility**: Các cột có tương thích để merge không?
3. **Merge impact**: Gộp chung sẽ tạo ra bao nhiêu % NaN?
4. **Modeling strategy**: Nên merge hay giữ riêng cho từng loại thiết bị?

---